# Assignment 2: Using Gurobipy

In [1]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 37.2 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import math

## Exercise 1: Lot-sizing

In [3]:
# Input data
demands = [ [6048, 5343, 3240, 4598, 8172, 6341, 3834, 2619, 2128, 2943],
            [873, 1123, 1106, 678, 858, 939, 1060, 730, 826, 1471],
            [465, 1376, 636, 514, 1083, 640, 977, 1022, 938, 879],
            [183, 156, 48, 27, 159, 105, 122, 62, 102, 19]]

setup_cost = [300, 50, 50, 50, 50]
holding_cost = [0.001, 0.01, 0.01, 0.1]

M = 100000

num_items = len(demands)
num_periods = len(demands[0])

In [4]:
model = gp.Model("Wagner-Whitin")

lotsize = model.addVars(num_items, num_periods, vtype=GRB.INTEGER, name="lotsize")
setup = model.addVars((num_items+1), num_periods, vtype=GRB.BINARY, name="setup")
inventories = model.addVars(num_items, num_periods, name="inventories")

model.setObjective(quicksum(setup[item+1, period]*setup_cost[item+1]+holding_cost[item]*inventories[item, period]
                            for item in range(num_items)
                            for period in range(num_periods))
                  + quicksum(setup[0, period]*setup_cost[0]
                             for period in range(num_periods)), GRB.MINIMIZE)

# Inventory balance constraints
model.addConstrs(inventories[item, 0] == lotsize[item, 0]-demands[item][0]
                for item in range(num_items))
model.addConstrs(inventories[item, period] == lotsize[item, period]-demands[item][period]+inventories[item, period-1]
                 for item in range(num_items)
                 for period in range(1, num_periods))

# Logic constraints
model.addConstrs(lotsize[item, period] <= M*setup[item+1, period]
                 for item in range(num_items)
                 for period in range(num_periods))
model.addConstrs(setup[item+1, period] <= setup[0, period]
                 for item in range(num_items)
                 for period in range(num_periods))

Restricted license - for non-production use only - expires 2024-10-28


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


In [5]:
# Run model
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 120 rows, 130 columns and 276 nonzeros
Model fingerprint: 0xdeac28a2
Variable types: 40 continuous, 90 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-03, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 8e+03]
Found heuristic solution: objective 4001.9180000
Presolve removed 16 rows and 17 columns
Presolve time: 0.00s
Presolved: 104 rows, 113 columns, 240 nonzeros
Variable types: 0 continuous, 113 integer (45 binary)
Found heuristic solution: objective 4001.6960000

Root relaxation: objective 7.176665e+02, 60 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



In [6]:
# Print results
for variable in model.getVars():
    if variable.varName.startswith("lotsize"):
        print(variable.varName+": "+str(variable.x))

print("Objective: "+str(model.objVal))

lotsize[0,0]: 19229.0
lotsize[0,1]: 0.0
lotsize[0,2]: -0.0
lotsize[0,3]: 0.0
lotsize[0,4]: 26037.0
lotsize[0,5]: 0.0
lotsize[0,6]: 0.0
lotsize[0,7]: 0.0
lotsize[0,8]: 0.0
lotsize[0,9]: 0.0
lotsize[1,0]: 3780.0
lotsize[1,1]: 0.0
lotsize[1,2]: -0.0
lotsize[1,3]: -0.0
lotsize[1,4]: 5884.0
lotsize[1,5]: 0.0
lotsize[1,6]: 0.0
lotsize[1,7]: 0.0
lotsize[1,8]: 0.0
lotsize[1,9]: 0.0
lotsize[2,0]: 2991.0
lotsize[2,1]: 0.0
lotsize[2,2]: 0.0
lotsize[2,3]: 0.0
lotsize[2,4]: 5539.0
lotsize[2,5]: 0.0
lotsize[2,6]: 0.0
lotsize[2,7]: 0.0
lotsize[2,8]: 0.0
lotsize[2,9]: 0.0
lotsize[3,0]: 414.0
lotsize[3,1]: 0.0
lotsize[3,2]: -0.0
lotsize[3,3]: 0.0
lotsize[3,4]: 569.0
lotsize[3,5]: 0.0
lotsize[3,6]: 0.0
lotsize[3,7]: 0.0
lotsize[3,8]: 0.0
lotsize[3,9]: 0.0
Objective: 1600.5500000000002


## Exercise 2: Rolling horizon planning with forecast updates

In [7]:
def rolling_horizon(horizon):
  demands = [66, 78, 57, 34, 5, 118, 76, 61, 23, 31, 68, 48]
  periods = len(demands)
  runs = periods-horizon+1
  smoothing = 0.3
  initial_inv = 30
  setup_cost = 450
  holding_cost = 2
  backorder_cost = 20

  BIG_M = sum(demands[t] for t in range(periods))
  quantities = np.zeros(periods)

  for k in range(runs):
      forecast = [50]
      for i in range(1, horizon):
          forecast.append(smoothing*demands[k+i-1]+(1-smoothing)*forecast[i-1])
      print(f"forecast: {forecast}")
      # Create a new model in each iteration
      model = gp.Model()
      # Create variables
      lotsize = model.addVars(horizon, name="lotsize")
      inventory = model.addVars(horizon, name="inventory")
      setup = model.addVars(horizon, vtype=GRB.BINARY, name="setup")
      # Set objective
      model.setObjective(quicksum(setup[t]*setup_cost+holding_cost*inventory[t]
                        for t in range(horizon)), GRB.MINIMIZE)
      # Add constraints
      model.addConstr(inventory[0] == initial_inv+lotsize[0]-forecast[0])
      model.addConstrs(inventory[t-1]+lotsize[t]-forecast[t] == inventory[t] for t in range(1, horizon))
      model.addConstrs(lotsize[t] <= BIG_M*setup[t] for t in range(horizon))
      # Optimize model
      model.optimize()
      variables = model.getVars()
      quantities[k] = variables[0].x
      initial_inv = initial_inv+variables[0].x-demands[k]
      if k == runs-1:
          for t in range(horizon-1):
              # Note that the first variables are lotsizes and 0 is already assigned
              quantities[runs+t] = variables[t+1].x
      model.dispose()

  print("\n\n\n")
  print("Quantities: "+str(quantities))
  cost = 0
  inv = 0
  for t in range(periods):
      if quantities[t] > 0:
          cost = cost+setup_cost
      inv = inv+quantities[t]-demands[t]
      if inv > 0:
          cost = cost+holding_cost*inv
      else:
          cost = cost-backorder_cost*inv
  print("Rolling Horizon Cost: "+str(cost))

In [8]:
rolling_horizon(horizon=4)

forecast: [50, 54.8, 61.75999999999999, 60.331999999999994]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 19 nonzeros
Model fingerprint: 0x4c12db0d
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 6e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 15 nonzeros
Variable types: 7 continuous, 3 integer (3 binary)
Found heuristic solution: objective 1800.0000000

Root relaxation: objective 5.697014e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

In [9]:
rolling_horizon(horizon=6)

forecast: [50, 54.8, 61.75999999999999, 60.331999999999994, 52.43239999999999, 38.20267999999999]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 18 columns and 29 nonzeros
Model fingerprint: 0xbddb5793
Variable types: 12 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 6e+01]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 22 nonzeros
Variable types: 9 continuous, 5 integer (5 binary)
Found heuristic solution: objective 2700.0000000

Root relaxation: objective 8.887089e+02, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

In [10]:
rolling_horizon(horizon=8)

forecast: [50, 54.8, 61.75999999999999, 60.331999999999994, 52.43239999999999, 38.20267999999999, 62.14187599999999, 66.29931319999999]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 24 columns and 39 nonzeros
Model fingerprint: 0x3248fd45
Variable types: 16 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 7e+01]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 13 rows, 20 columns, 32 nonzeros
Variable types: 13 continuous, 7 integer (7 binary)
Found heuristic solution: objective 3600.0000000

Root relaxation: objective 1.107429e+03, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

## Exercise 3: VRP

In [11]:
def vrp_index_2(d, K):
  n = len(d)-1
  Q = 4
  T = 16
  b = [0]
  for i in range(n):
    b.append(1)
  print(b)

  model = gp.Model("Travelling Salesman Problem VRP Index 2")
  # Create variables
  travel = model.addVars(n+1, n+1, vtype=GRB.BINARY, name="travel")
  y = model.addVars(n+1, n+1, name="y")
  # Set objective
  model.setObjective(quicksum(d[city_1][city_2] * travel[city_1, city_2]
                    for city_1 in range(n+1)
                    for city_2 in range(n+1)),
                     GRB.MINIMIZE)

  # Ensure tour
  model.addConstrs((quicksum(travel[city_1, city_2] for city_2 in range(n+1)) == 1
                      for city_1 in range(1, n+1)), name="arrival")
  model.addConstrs((quicksum(travel[city_1, city_2] for city_1 in range(n+1)) == 1
                      for city_2 in range(1, n+1)), name="departure")

  # Total number of tours  
  model.addConstr(quicksum(travel[city_1, 0] for city_1 in range(n+1)) == 
                  quicksum(travel[0, city_2] for city_2 in range(n+1)))
  model.addConstr(quicksum(travel[0, city_2] for city_2 in range(n+1)) <= K)

  # Demand at location j
  model.addConstrs(quicksum(y[city_1, city_2]-y[city_2, city_1] for city_1 in range(n+1)) == b[city_2]
                   for city_2 in range(1, n+1))

  # Total demand
  model.addConstr(quicksum(y[0, city_1] for city_1 in range(n+1)) == 
                  quicksum(b[city_2] for city_2 in range(1, n+1)))
  
  # Transportation capacity
  model.addConstrs((b[city_2]*travel[city_1, city_2] <= y[city_1, city_2])
                    for city_1 in range(n+1)
                    for city_2 in range(n+1))
  model.addConstrs((y[city_1, city_2] <= (Q-b[city_1])*travel[city_1, city_2])
                    for city_1 in range(n+1)
                    for city_2 in range(n+1))

  # Empty vehicle at the end
  model.addConstrs((y[city, 0] == 0) for city in range(n+1))

  # Run optimization
  model.optimize()

  # Get solution
  solution = model.getAttr('x', travel)
  for city_1 in range(n+1):
      for city_2 in range(n+1):
          if solution[city_1, city_2] > 0:
              print('%s -> %s: %g' % (city_1, city_2, solution[city_1, city_2]))
  print("Objective: "+str(model.objVal))

In [12]:
def vrp_index_3(d, K):
  n = len(d)-1
  Q = 4
  T = 16
  b = [0]
  for i in range(n):
    b.append(1)
  print(b)

  model = gp.Model("Travelling Salesman Problem VRP Index 3")
  # Create variables
  z = model.addVars(n+1, name="auxiliary_var")
  travel = model.addVars(n+1, n+1, K, vtype=GRB.BINARY, name="travel")
  y = model.addVars(n+1, K, vtype=GRB.BINARY, name="y")
  # Set objective
  model.setObjective(quicksum(d[city_1][city_2] * travel[city_1, city_2, k]
                    for city_1 in range(n+1)
                    for city_2 in range(n+1)
                    for k in range(K)),
                     GRB.MINIMIZE)

  # Arrival constraints
  model.addConstrs((quicksum(travel[city_1, city_2, k] for city_2 in range(n+1)) == y[city_1, k]
                      for city_1 in range(1, n+1)
                      for k in range(K)), name="arrival")
  # Departure constraints
  model.addConstrs((quicksum(travel[city_1, city_2, k] for city_1 in range(n+1)) == y[city_2, k]
                      for city_2 in range(1, n+1)
                      for k in range(K)), name="departure")

  # Subtourelimination constraints
  for city_1 in range(1, n+1):
      for city_2 in range(1, n+1):
          if city_1 != city_2:
              model.addConstr(z[city_1] - z[city_2] +
                              n * quicksum(travel[city_1, city_2, k] for k in range(K)) <= n - 1)
  
  # Cluster
  model.addConstrs(quicksum(y[city, k] for k in range(K)) == 1
                   for city in range(1, n+1))
  model.addConstr(quicksum(y[0, k] for k in range(K)) == K)

  # Capacities
  model.addConstrs(quicksum(b[city]*y[city, k] for city in range(1, n+1)) <= Q
                   for k in range(K))
  model.addConstrs(quicksum(d[city_1][city_2]*travel[city_1, city_2, k]
                            for city_1 in range(n+1)
                            for city_2 in range(n+1)) <= T
                   for k in range(K))

  # Run optimization
  model.optimize()

  # Get solution
  solution = model.getAttr('x', travel)
  for k in range(K):
    print(f"k = {k}")
    for city_1 in range(n+1):
        for city_2 in range(n+1):
            if solution[city_1, city_2, k] > 0:
                print('%s -> %s: %g' % (city_1, city_2, solution[city_1, city_2, k]))
  print("Objective: "+str(model.objVal))


In [13]:
def get_tour_cost(tour, d):
  n_cities = len(tour)
  if n_cities == 0:
    return 0
  cost = d[0][tour[0]] + d[tour[n_cities-1]][0]
  for i in range(1, n_cities):
    u = tour[i-1]
    v = tour[i]
    cost += d[u][v]
  return cost

def improve(tour_idx_1, tour_1, tour_idx_2, tour_2, pop_start, push_start, d, Q, T, tours):
  cost_old = get_tour_cost(tour_1, d) + get_tour_cost(tour_2, d)
  tour_1_new = []
  tour_2_new = []
  n_cities_1 = len(tour_1)
  n_cities_2 = len(tour_2)
  pivot_city_index = 0 if pop_start else n_cities_1-1
  pivot_city = tour_1[pivot_city_index]
  for city in tour_1:
    if city != pivot_city:
      tour_1_new.append(city)
  if push_start:
    tour_2_new.append(pivot_city)
  for city in tour_2:
    tour_2_new.append(city)
  if not push_start:
    tour_2_new.append(pivot_city)
  if max(len(tour_1_new), len(tour_2_new)) > Q:
    return 0, None
  cost_new_1 = get_tour_cost(tour_1_new, d)
  cost_new_2 = get_tour_cost(tour_2_new, d)
  cost_new = cost_new_1 + cost_new_2
  if cost_new >= cost_old or max(cost_new_1, cost_new_2) > T:
    return 0, None
  new_tours = []
  for tour_idx, tour in enumerate(tours):
    if tour_idx != tour_idx_1 and tour_idx != tour_idx_2:
      new_tours.append(tour)
    elif tour_idx_1 == tour_idx:
      if tour_1_new:
        new_tours.append(tour_1_new)
    elif tour_idx_2 == tour_idx:
      if tour_2_new:
        new_tours.append(tour_2_new)
  return cost_old-cost_new, new_tours

def vrp_savings_method(d):
  n = len(d)-1
  Q = 4
  T = 16
  tours = []
  total_cost = 0
  for i in range(1,n+1):
    total_cost += d[0][i] + d[i][0]
    tours.append([i])
  while True:
    n_tours = len(tours)
    best_tours = None
    best_improvement = 0
    for tour_idx_1, tour_1 in enumerate(tours):
      for tour_idx_2, tour_2 in enumerate(tours):
        for pop_start in (True, False):
          for push_start in (True, False):
            improvement, new_tours = improve(tour_idx_1, tour_1, tour_idx_2, tour_2, pop_start, push_start, d, Q, T, tours)
            if improvement > best_improvement:
              best_improvement = improvement
              best_tours = new_tours
    if best_improvement > 0:
      total_cost -= best_improvement
      tours = best_tours
    else:
      break
  print(total_cost)
  for tour in tours:
    print(tour)

In [14]:
BIG_COST = 1000000
coordinates = [[0, 0], [4, 1], [1, 2], [0, 5], [-3, 3], [-2, 1], [-5, 1], [-5, -1], [-1, -3], [3, -2], [6, -1]]
num_cities = len(coordinates)
d = []
for i, source in enumerate(coordinates):
  d_src = []
  for j, sink in enumerate(coordinates):
    if i == j:
      d_src.append(BIG_COST)
    else:
      d_x = source[0]-sink[0]
      d_y = source[1]-sink[1]
      d_src.append(math.sqrt(d_x*d_x+d_y*d_y))
  d.append(d_src)
  print(d_src)

[1000000, 4.123105625617661, 2.23606797749979, 5.0, 4.242640687119285, 2.23606797749979, 5.0990195135927845, 5.0990195135927845, 3.1622776601683795, 3.605551275463989, 6.082762530298219]
[4.123105625617661, 1000000, 3.1622776601683795, 5.656854249492381, 7.280109889280518, 6.0, 9.0, 9.219544457292887, 6.4031242374328485, 3.1622776601683795, 2.8284271247461903]
[2.23606797749979, 3.1622776601683795, 1000000, 3.1622776601683795, 4.123105625617661, 3.1622776601683795, 6.082762530298219, 6.708203932499369, 5.385164807134504, 4.47213595499958, 5.830951894845301]
[5.0, 5.656854249492381, 3.1622776601683795, 1000000, 3.605551275463989, 4.47213595499958, 6.4031242374328485, 7.810249675906654, 8.06225774829855, 7.615773105863909, 8.48528137423857]
[4.242640687119285, 7.280109889280518, 4.123105625617661, 3.605551275463989, 1000000, 2.23606797749979, 2.8284271247461903, 4.47213595499958, 6.324555320336759, 7.810249675906654, 9.848857801796104]
[2.23606797749979, 6.0, 3.1622776601683795, 4.472135

In [15]:
vrp_index_2(d, K=3)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 286 rows, 242 columns and 946 nonzeros
Model fingerprint: 0xd06d1b78
Variable types: 121 continuous, 121 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 53 rows and 32 columns
Presolve time: 0.01s
Presolved: 233 rows, 210 columns, 830 nonzeros
Variable types: 100 continuous, 110 integer (110 binary)
Found heuristic solution: objective 42.1971219

Root relaxation: objective 3.851661e+01, 136 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

In [16]:
vrp_index_3(d, K=3)

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 167 rows, 407 columns and 1596 nonzeros
Model fingerprint: 0xa9944d85
Variable types: 11 continuous, 396 integer (396 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e+00, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 11 rows and 47 columns
Presolve time: 0.01s
Presolved: 156 rows, 360 columns, 1500 nonzeros
Variable types: 10 continuous, 350 integer (350 binary)

Root relaxation: objective 2.931023e+01, 139 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   29.31023    0   28          -   29.31023  

In [17]:
vrp_savings_method(d)

44.84184523733969
[2, 3]
[5, 4, 6, 7]
[8]
[1, 10, 9]


## Exercise 4: Fixed Charge Transportation Problem with Benders Decomposition

In [18]:
# Please insert your code here

## Exercise 5: Stochastic Programming

In [19]:
# Please insert your code here